In [85]:
"""
    Maps Standings and Fixtures for building the Training Data
"""

'\n    Maps Standings and Fixtures for building the Training Data\n'

In [44]:
import json
import os
import pandas as pd
from pandas import json_normalize

In [45]:
# Historicity: multi season directory
year_window = 1
start_season = 2020
seasons = list(range(start_season, start_season - year_window, -1))
print(seasons)

[2020]


In [46]:
# Multi League directory
country = 'ES'
dirCountry = '../' + country + '/'

In [47]:
league = '140'
season = '2020'

In [37]:
# League directory
dirName = dirCountry + league + '/'

In [48]:
# The last week played
curr_week = 5

In [79]:
df = pd.DataFrame()
standings = None
curr_standings = None
away_team_stands = None
fixture = None
# This will prevent duplicated fixtures
visited_fixtures = []
# This will help to map to standings to week: team : stands
visited_standings = {}
"""visited_standings['1'] = {}"""
# Easy access to last standings
current_standings = {}

# Principal leagues are sorted ascendent
dirLeague = sorted(os.listdir(dirCountry))

for league in dirLeague:
    if league == ".DS_Store":
        continue
    if "csv" in league:
        continue
    if ".json" in league:
        continue
    for season in seasons:
        dirName = dirCountry + league + '/' + str(season) + '/' 
        for subdir in os.listdir(dirName):
            if subdir == ".DS_Store":
                continue
            if subdir == "teams.json":
                continue
            if ".csv" in subdir:
                continue
            team_id = subdir
            # Entering directory of a team
            dirTeam = dirName + team_id + '/'
            
            # Initializes all first standings
            """
            if team_id not in visited_standings['1']:
                json_standings = dirTeam + team_id + '_' + str(1) + '.json'
                with open(json_standings) as f:
                    first_standings = json.load(f)
                    visited_standings['1'][team_id] = first_standings
            """
            
            # Initializes all current standings
            c_w = list(range(curr_week, -1, -1))
            i_w = 0
            while team_id not in current_standings:
                we = c_w[i_w]
                json_standings = dirTeam + team_id + '_' + str(we) + '.json'
                try:
                    with open(json_standings) as f:
                        curr_standings = json.load(f)
                        current_standings[team_id] = curr_standings
                except:
                    print("id= %s, w=%s" % (team_id, str(w)))
                i_w = i_w + 1
            curr_standings = current_standings[team_id]
            
            # Gets standings from a team if they have not being added
            for week in list(range(0, 39, 1)):
                week = str(week)
                if week not in visited_standings:
                    visited_standings[week] = {}
                    if team_id not in visited_standings[week]:
                        try:
                            json_standings = dirTeam + team_id + '_' + week + '.json'
                            with open(json_standings) as f:
                                standings = json.load(f)
                                visited_standings[week][team_id] = standings
                        except:
                            visited_standings[week][team_id] = curr_standings
                            standings = curr_standings

            # Directory of team in home
            dirHome = dirTeam + 'home/'
            for file in os.listdir(dirHome):
                # Check if fixture hasn't being added to dataframe
                if file not in visited_fixtures:
                    path = dirHome + file
                    with open(path) as f:
                        fixture = json.load(f)
                        # Adds id of fixture as "id_fixture.json"
                        visited_fixtures.append(file)
                        # Next, get away team's external info
                        away_team = str(fixture["team_away"]["id"])
                        w = fixture["week"]
                        w = ''.join(filter(str.isdigit, w))
                        # Gets standings from away_team
                        """if w == '1':
                            try:
                                fileAwayTeam = dirName + away_team + '/' + away_team + '_' + str(1) + '.json'
                                with open(fileAwayTeam) as f:
                                    away_team_stands = json.load(f)
                                    visited_standings['1'][away_team] = away_team_stands
                            except:
                                print(fileAwayTeam)
                                print("Something rare")
                        """
                        if away_team in visited_standings[w]:
                            away_team_stands = visited_standings[w][away_team]
                        else:
                            try:
                                fileAwayTeam = dirName + away_team + '/' + away_team + '_' + week + '.json'
                                with open(fileAwayTeam) as f:
                                    away_team_stands = json.load(f)
                                    visited_standings[w][away_team] = away_team_stands
                            except:
                                i_w = 0
                                while away_team not in current_standings:
                                    we = c_w[i_w]
                                    fileAwayTeam = dirName + away_team + '/' + away_team + '_' + str(we) + '.json'
                                    try:
                                        with open(fileAwayTeam) as f:
                                            away_team_stands = json.load(f)
                                            visited_standings[w][away_team] = away_team_stands
                                            current_standings[away_team] = away_team_stands
                                    except:
                                        print("id= %s, w=%s" % (away_team, str(we)))
                                    i_w = i_w + 1
                        # Home team standings
                        if team_id in visited_standings[w]:
                            standings = visited_standings[w][team_id]
                        else:
                            standings = current_standings[team_id]
                        # Gets away current standings if fixtures is set to null
                        if fixture["goals_away"] is None:
                            away_team_stands = current_standings[away_team]
                            # Changes if game to be played
                            standings = current_standings[team_id]
                        else:
                            if away_team in visited_standings[w]:
                                away_team_stands = visited_standings[w][away_team]
                            else:
                                away_team_stands = current_standings[away_team]
                    # Three main jsons to a dataframe
                    # standings "this is the home team"
                    # away_team_stands
                    # the fixture itself
                    # The premise that away standings from home team aren't relevant
                    try:
                        #del standings['stats_away']
                        fixture.update({"team_home": standings})
                    except:
                        print("Couldn't delete stats away")
                    # The premise that home standings from away team aren't relevant
                    try:
                        #del away_team_stands['stats_home']
                        fixture.update({"team_away": away_team_stands})
                    except:
                        print("Couldn't delete stats home")
                    # Adding league
                    fixture.update({"league": int(league)})
                    # Adding season
                    fixture.update({"season": season})
                    df = df.append(json_normalize(fixture))
                else:
                    continue

id= 797, w=5
id= 797, w=5
id= 539, w=5
id= 541, w=5
id= 531, w=5
id= 727, w=5
id= 536, w=5
id= 529, w=2
id= 529, w=2
id= 546, w=5
id= 530, w=2
id= 530, w=2


In [80]:
visited_standings['1']

{'797': {'team': {'id': '797', 'name': 'Elche'},
  'rank': 20,
  'points': 0,
  'goals_diff': -3,
  'form': 'L',
  'description': 'Relegation - LaLiga2',
  'group': 'Primera División',
  'stats_home': {'played': 1,
   'wins': 0,
   'draws': 0,
   'lose': 1,
   'goals_for': 0.0,
   'avg_goals_for': 0.0,
   'goals_against': 3.0,
   'avg_goals_against': 3.0,
   'clean_sheet': 0,
   'failed_to_score': 1,
   'streaks': {'wins': 0,
    'draws': 0,
    'loses': 1,
    'best_win': None,
    'best_lose': '0-3',
    'best_goals_for': 0,
    'best_goals_against': 3}},
  'stats_away': {'played': 0,
   'wins': 0,
   'draws': 0,
   'lose': 0,
   'goals_for': 0.0,
   'avg_goals_for': 0.0,
   'goals_against': 0.0,
   'avg_goals_against': 0.0,
   'clean_sheet': 0,
   'failed_to_score': 0,
   'streaks': {'wins': 0,
    'draws': 0,
    'loses': 1,
    'best_win': None,
    'best_lose': None,
    'best_goals_for': 0,
    'best_goals_against': 0}}},
 '539': {'team': {'id': '539', 'name': 'Levante'},
  'ran

In [81]:
df.head(10)

,id,season,week,goals_home,goals_away,league,team_home.team.id,team_home.team.name,team_home.rank,team_home.points,...,stats_away.offside,stats_away.possession,stats_away.c_yellow,stats_away.c_red,stats_away.saves,stats_away.p_total,stats_away.p_accurate,stats_away.p_percentage,stats_home,stats_away
0,605114,2020,5,0,0,140,797,Elche,16,4,...,2,49%,2,None,None,473.0,408.0,86%,NaN,NaN
0,605094,2020,3,0,3,140,797,Elche,17,3,...,2,60%,1,None,2,492.0,423.0,86%,NaN,NaN
0,605083,2020,2,2,1,140,533,Villarreal,4,8,...,5,43%,1,None,5,305.0,216.0,71%,NaN,NaN
0,605073,2020,1,1,1,140,533,Villarreal,4,8,...,4,35%,1,None,3,280.0,210.0,75%,NaN,NaN
0,605104,2020,4,3,1,140,533,Villarreal,4,8,...,2,42%,2,None,2,378.0,285.0,75%,NaN,NaN
0,605071,2020,1,4,2,140,532,Valencia,8,7,...,1,56%,4,None,4,460.0,394.0,86%,NaN,NaN
0,605092,2020,3,1,1,140,532,Valencia,8,7,...,3,52%,1,None,1,397.0,328.0,83%,NaN,NaN
0,605112,2020,5,0,2,140,532,Valencia,8,7,...,None,54%,2,None,2,454.0,379.0,83%,NaN,NaN
0,605088,2020,3,2,3,140,543,Real Betis,2,9,...,8,55%,2,None,3,539.0,466.0,86%,NaN,NaN
0,605075,2020,2,2,0,140,543,Real Betis,2,9,...,4,44%,None,None,4,390.0,295.0,76%,NaN,NaN


In [82]:
df.shape

(47, 124)

In [83]:
# csv to save
file_title = str(season) + '_' + str(curr_week) + '.csv'
csv_file = dirName + file_title
print(file_title)
print(csv_file)

2020_5.csv
../ES/140/2020/2020_5.csv


In [85]:
df.to_csv(csv_file)